In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 

In [518]:
# Objective:
# predict total sales for every product and store in the next month
# Create naive benchmark, where prediction is simply the previous months sale
# test.csv: the test set. You need to forecast the sales for these shops and products for November 2015.
# Gets 1.16777 on LB which places at 113th place as of 1/10/2018

In [447]:
sample = pd.read_csv('sample_submission.csv')
test = pd.read_csv('test.csv')

In [448]:
sample.head(3)

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5


In [449]:
test_orig = test.copy()
test['date_block_num'] = np.ones(len(test.index))* 34
test['item_cnt_day'] = np.zeros(len(test.index))
cols = test.columns.tolist()
print(cols)
cols = cols[:1]+ cols[-2:-1]+cols[1:3]+cols[-1:]
print(cols)
test = test[cols]
ID = test['ID']
del test['ID']
test.head()


['ID', 'shop_id', 'item_id', 'date_block_num', 'item_cnt_day']
['ID', 'date_block_num', 'shop_id', 'item_id', 'item_cnt_day']


,date_block_num,shop_id,item_id,item_cnt_day
0,34.0,5,5037,0.0
1,34.0,5,5320,0.0
2,34.0,5,5233,0.0
3,34.0,5,5232,0.0
4,34.0,5,5268,0.0


In [450]:
transactions = pd.read_csv('sales_train.csv.gz')
items = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')

print(transactions.shape)
print(items.shape)
print(item_categories.shape)
print(shops.shape)
transactions.head(3)

(2935849, 6)
(22170, 3)
(84, 2)
(60, 2)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
1,03.01.2013,0,25,2552,899.0,1.0
2,05.01.2013,0,25,2552,899.0,-1.0


In [451]:
#transactions[['Day','Month','Year']]=transactions.date.str.split('.',expand=True)
train = transactions.copy()
del train['date']
del train['item_price']

In [452]:
train.head()

,date_block_num,shop_id,item_id,item_cnt_day
0,0,59,22154,1.0
1,0,25,2552,1.0
2,0,25,2552,-1.0
3,0,25,2554,1.0
4,0,25,2555,1.0


In [488]:
df = pd.concat([train,test])

In [489]:
df.tail()

,date_block_num,shop_id,item_id,item_cnt_day
214195,34.0,45,18454,0.0
214196,34.0,45,16188,0.0
214197,34.0,45,15757,0.0
214198,34.0,45,19648,0.0
214199,34.0,45,969,0.0


In [490]:

df_lag = df.copy()

In [491]:
df_lag.date_block_num+=1


In [492]:
df_lag.columns = ['date_block_num','shop_id','item_id','pp_item_cnt_month_lag_1']

In [493]:
df = df.merge(df_lag, on=['date_block_num','shop_id','item_id'], how='left').fillna(0.0)

In [499]:
test_pred = df[df['date_block_num'] == 34]
test_pred.head()
print(len(test_pred))

235052


In [500]:
m_sales = test_pred.groupby(['date_block_num','shop_id','item_id'])
test_pred_m = pd.DataFrame(m_sales.pp_item_cnt_month_lag_1.sum())


In [501]:
test_pred_m.reset_index(inplace=True) 


In [503]:
test_pred_m.tail(3)

,date_block_num,shop_id,item_id,pp_item_cnt_month_lag_1
214197,34.0,59,22164,0.0
214198,34.0,59,22166,0.0
214199,34.0,59,22167,0.0


In [505]:

del test_pred_m['date_block_num']
test_pred_m.head()

,shop_id,item_id,pp_item_cnt_month_lag_1
0,2,30,0.0
1,2,31,1.0
2,2,32,0.0
3,2,33,0.0
4,2,38,0.0


In [476]:
test_orig.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [506]:
submission = test_orig.copy()
submission_temp = submission.merge(test_pred_m, on=['shop_id','item_id'], how='left')

In [507]:
submission_temp.head()

,ID,shop_id,item_id,pp_item_cnt_month_lag_1
0,0,5,5037,0.0
1,1,5,5320,0.0
2,2,5,5233,1.0
3,3,5,5232,0.0
4,4,5,5268,0.0


In [509]:
final_submission = sample.copy()
final_submission.item_cnt_month = submission_temp.pp_item_cnt_month_lag_1.clip(0,20)
final_submission.head()

,ID,item_cnt_month
0,0,0.0
1,1,0.0
2,2,1.0
3,3,0.0
4,4,0.0


In [515]:
final_submission.to_csv('prev_month_benchmark.csv',index=False)

In [516]:
sample.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [517]:
sub = pd.read_csv('prev_month_benchmark.csv')
sub.head()

,ID,item_cnt_month
0,0,0.0
1,1,0.0
2,2,1.0
3,3,0.0
4,4,0.0
